In [495]:
# Date: 7/17/18
# Author: Joey Li
#
# This project can theoretically correct errors on the [[5,1,3]] code. In particular, unflagged (non-fault-tolerant)
# correction should work. 
#
# Much of the framework was also built for correcting errors fault-tolerantly using flagged syndrome extraction as 
# outlined in https://arxiv.org/pdf/1705.02329.pdf. However, current publicly available quantum computers such
# as those available through IBM Q and Rigetti Forest are not capable of performing measurements in the middle
# of a circuit, a crucial component to error correction. This project is currently not being maintained.

from qiskit import QuantumProgram, QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import qiskit
import random, pickle, time, datetime, xlsxwriter
import scipy.io as spio
import importlib
import circuit_builder
importlib.reload(circuit_builder)
import encoding_schemes
importlib.reload(encoding_schemes)
import operator

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
    print()
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set 
QX_URL = "https://quantumexperience.ng.bluemix.net/api"
QX_TOKEN = "06704f99fc6dd68ab5ae2663f0861723406de8b4553cacdabdf500292c937717442daa90c2ffa19109cfb9114adb7c87711a08860d34d1da9332ac664e4e24a7"

#register token and url
register(QX_TOKEN, QX_URL)

# query for available backends
print("Available backends: ", available_backends())

# Specify this to be local_qasm_simulator when debugging
backend = "ibmqx5"




Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\qecc_benchmark\Qconfig.py.

Available backends:  ['ibmqx5', 'local_qasm_simulator', 'ibmqx2', 'local_clifford_simulator', 'ibmq_qasm_simulator', 'local_statevector_simulator', 'ibmqx4', 'local_unitary_simulator']


In [513]:
# Grabs quantum register k from qc
def getQReg(qc,k):   
    desired = list(qc.get_qregs().items())
    return desired[k][1]

# Grabs classical register k from qc
def getCReg(qc,k):   
    desired = list(qc.get_cregs().items())
    return desired[k][1]  

# This method is used to return syndrome measurements
def getAnc(qc,backend):
    job = execute(qc,backend)
    result = job.result()
    data = result.get_counts()
    #print(data)
    
    # Generates dictionary of syndromes with frequencies
    ancFreqs = {}
    for key in data:
        print(key)
        arr = key.split()
        ancFreqs[arr[0]] = data[key]
        
    # Obtains most likely syndrome
    ancBits = max(ancFreqs.items(), key=operator.itemgetter(1))[0]
    return ancBits

    
class FiveOneThree:
    # Number of qubits
    n = 5
    
    # Defines code space
    validOutputs = ["00000", "10010", "01001", "10100", "01010", "11011", "00110",
                    "11000", "11101", "00011", "11110", "01111", "10001", "01100",
                    "10111", "00101", "11111", "01101", "10110", "01011", "10101",
                    "00100", "11001", "00111", "00010", "11100", "00001", "10000",
                    "01110", "10011", "01000", "11010"]

    # Read things right to left
    ind = {6:0, 7:1}
    
    # Sets mapping of qubit numbering so that qubits numbered "1 to n" appear
    # in that order left to right
    perm = {}
    
    # How many registers used so far?
    ancCounter = 0
    
    # Standard setperm method
    def setPerm(self):
        n = self.n
        for i in range(n):
            self.perm[n-i] = i

    
    # prepare a state of all zeros
    def prepareZeros(self,qc):
        
        # First time we add error correction
        # 2 ancilla qubits used to check
        qr = QuantumRegister(5)
        cr = ClassicalRegister(5)
        qc.add(qr)
        qc.add(cr)
        
        # Removed for sake of quantum register efficiency (no flags in register)
        #anc = QuantumRegister(2)
        
        # Replaced temporarily
        reg = ClassicalRegister(1)
        #reg = ClassicalRegister(2)
        #qc.add(anc)
        qc.add(reg)
        
        # Whether state preparation was correct
        correct = False
        
        while not correct:
            # Actual State Preparation
            # Start by preparing +
            allPlus = ["H 1 2 3 4 5", "CZ 1 2", "CZ 3 4", 
                      "CZ 2 3", "CZ 4 5", "CZ 1 5"]
            
            circuit_builder.buildCirc(qc,allPlus,perm)
            
            # THIS MAKES IT FAULT TOLERANT AND SHOULD COME BACK LATER
            
            '''
            # Check state preparation
            
            # Check 1
            qc.reset(anc)
            
            qc.h(anc[0])
            qc.cx(anc[0],qr[perm[1]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[2]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[5]])
            
            qc.h(anc[0])
            qc.barrier()
            qc.measure(anc[0],reg[0])
            qc.barrier()
            qc.measure(anc[1],reg[1])
            qc.barrier()
            
            if getAnc(qc,backend) != "00":
                print("FAILURE IN STATE PREP")
                return "FAILURE"
            
            # Check 2
            qc.reset(anc)
            qc.h(anc[0])
            qc.cz(anc[0],qr[perm[1]])
            qc.cx(anc[0],anc[1])
            qc.cx(anc[0],qr[perm[2]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[3]])
            qc.h(anc[0])
            qc.barrier(anc)
            qc.barrier(qr)
            qc.measure(anc[0],reg[0])
            qc.measure(anc[1],reg[1])
            
            if getAnc(qc,backend) != "00":
                print("FAILURE IN STATE PREP")
                return "FAILURE"
                
            # Check 3
            qc.reset(anc)
            qc.h(anc[0])
            qc.cx(anc[0],qr[perm[4]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[3]])
            qc.cx(anc[0],anc[1])
            qc.cz(anc[0],qr[perm[5]])
            qc.h(anc[0])
            qc.barrier(anc)
            qc.barrier(qr)
            qc.measure(anc[0],reg[0])
            qc.measure(anc[1],reg[1])
            
            if getAnc(qc,backend) != "00":
                print("FAILURE IN STATE PREP")
                return "FAILURE"
            '''
            correct = True
            
        
        
        # This is centralizing solution for transversal H
        # Converts to |0>
        # Try normalizing later
        
        transH = ["CX 3 2", "CX 5 1", "CX 5 4", "CX 3 4",
                 "CX 2 4", "CX 1 4", "H 1 2 3 4 5", "CZ 1 5",
                 "CZ 2 3", "H 1 2", "CX 2 4", "CX 1 4"]
        
        circuit_builder.buildCirc(qc, transH, perm)
        
        return "SUCCESS"
        
    # Sets logical input to |0> or |1>
    def setInput(self, qc, compiler):
        qr = getQReg(qc,0)
        
        if compiler == 0: 
            return
        elif compiler == 1:
            qc.x(qr)
        else:
            print("Invalid compiler")
        qc.barrier()

    # Altered to be usable without reset
    # Measure x from qubit a to target b
    def xMeas(self,qc,a,b):
        qr = getQReg(qc,0)
        anc = getQReg(qc,self.ancCounter)
        
        a = self.perm[a]
        b = self.ind[b]
        qc.h(qr[a])
        qc.cx(qr[a],anc[b])
        qc.h(qr[a])
    
    # Altered to be usable without reset
    # Measure z from qubit a to target b
    def zMeas(self,qc,a,b):
        qr = getQReg(qc,0)
        anc = getQReg(qc,self.ancCounter)
        
        a = self.perm[a]
        b = self.ind[b]
        qc.cx(qr[a],anc[b])

    # stab is stabilizer number, flag is boolean 
    def measureStab(self, qc, stab, flag):
        qr = getQReg(qc,0)
        reg = getCReg(qc,1)
        
        #Changed to work without possibility of resetting quantum registers
        #anc = getQReg(qc,1)
        anc = QuantumRegister(1)
        qc.add(anc)
        
        print("\nUSED AN ANCILLA")
        print("Quantum registers...")
        print(qc.get_qregs())
        print("Ancilla Register")
        print(anc)
        self.ancCounter +=1
        print(str(self.ancCounter)+" ancillas used so far")
        print("Measuring stabilizer "+str(stab))
        

        # stabilizer 0 is XZZXI
        if stab == 0:
            if flag:
                qc.h(anc[1])
            self.xMeas(qc,1,6)
            if flag:
                qc.cx(anc[1],anc[0])
            self.zMeas(qc,2,6)
            self.zMeas(qc,3,6)
            if flag:
                qc.cx(anc[1],anc[0])
            self.xMeas(qc,4,6)
                
        # stabilizer 1 is IXZZX
        elif stab == 1:
            if flag:
                qc.h(anc[1])
            self.xMeas(qc,2,6)
            if flag:
                qc.cx(anc[1],anc[0])
            self.zMeas(qc,3,6)
            self.zMeas(qc,4,6)
            if flag:
                qc.cx(anc[1],anc[0])
            self.xMeas(qc,5,6)
            
        # stabilizer 2 is XIXZZ
        elif stab == 2:
            if flag:
                qc.h(anc[1])
            self.xMeas(qc,1,6)
            if flag:
                qc.cx(anc[1],anc[0])
            self.zMeas(qc,4,6)
            self.zMeas(qc,5,6)
            if flag:
                qc.cx(anc[1],anc[0])
            self.xMeas(qc,3,6)
            
        # stabilizer 3 is ZXIXZ
        elif stab == 3:
            if flag:
                qc.h(anc[1])
            self.xMeas(qc,2,6)
            if flag:
                qc.cx(anc[1],anc[0])
            self.zMeas(qc,1,6)
            self.zMeas(qc,5,6)
            if flag:
                qc.cx(anc[1],anc[0])
            self.xMeas(qc,4,6)
            
            
        # Check syndrome            
        qc.barrier(anc)
        qc.measure(anc[0],reg[0])
        qc.barrier(anc)

        # Check flag
        if flag:
            qc.barrier(anc)
            qc.h(anc[1])
            qc.barrier(anc)
            qc.measure(anc[1],reg[1])

        synd = getAnc(qc,backend)

        print("Syndrome Measurement:" +str(synd))
        # synd is second, flag is first        
        return synd
    
    # This does straightforward fix of errors
    def fixError(self, qc, perm, syndrome):
        decoder = {
            "0000": [],
            "0001": ["X 1"],
            "1000": ["X 2"],
            "1100": ["X 3"],
            "0110": ["X 4"],
            "0011": ["X 5"],
            "1010": ["Z 1"],
            "0101": ["Z 2"],
            "0010": ["Z 3"],
            "1001": ["Z 4"],
            "0100": ["Z 5"],
            "1011": ["Y 1"],
            "1101": ["Y 2"],
            "1110": ["Y 3"],
            "1111": ["Y 4"],
            "0111": ["Y 5"]
        }
        correction = decoder[syndrome]
        print(correction)
        circuit_builder.buildCirc(qc, correction, perm)
    
    def unflagCorrect(self,qc,perm):
        syndrome = ""
        for i in range(4):
            m = scheme.measureStab(qc,i,False)
            syndrome += str(m[-1])
        scheme.fixError(qc,perm,syndrome)
        print(syndrome)
        return syndrome
    
    # Fix correlated errors indicated by flag
    def fixFlag(self,qc,perm, flagNum, syndrome):
        # Fix a flag obtained on XZZXI
        # Simplifications made in calculating fix
        fixZero = {
            "0000": [],
            "0100": ["Z 3", "X 4"],
            "1100": ["Y 4", "X 5"],
            "1001": ["X 1", "X 2"],
            "0001": ["X 1"],
            "0110": ["X 4"],
            "1010": ["X 3", "X 4"],
            "1000": ["Y 3", "X 4"]
        }
        # Fix a flag obtained on IXZZX
        # Simplifications made in calculating fix
        fixOne = {
            "0000": [],
            "0011": ["X 5"],
            "0100": ["X 2", "X 3"],
            "0101": ["X 4", "X 5"],
            "0110": ["X 1", "Y 5"],
            "1000": ["X 2"],
            "1010": ["Z 4", "X 5"],
            "1100": ["Y 4", "X 5"]
        }
        # Fix a flag obtained on XIXZZ
        # Simplifications made in calculating fix
        fixTwo = {
            "0000": [],
            "1100": ["X 3"],
            "1111": ["X 3", "X 5"],
            "1011": ["X 3", "Y 5"],
            "1000": ["X 3", "Z 5"],
            "1110": ["X 3", "X 4", "Z "],
            "0111": ["X 1", "X 4"],
            "0001": ["X 1"]
        }
        # fix a flag obtained on ZXIXZ
        # Simplifications made in calculating fix
        fixThree = {
            "0000": [],
            "0110": ["X 4"],
            "0101": ["X 4", "X 5"],
            "0001": ["X 4", "Y 5"],
            "0010": ["X 4", "Z 5"],
            "0011": ["Y 1", "X 2"],
            "1001": ["X 1", "X 2"],
            "1000": ["X 2"]
        }
        # For readability
        mapping = {
            0: fixZero,
            1: fixOne,
            2: fixTwo,
            3: fixThree
        }
        fixer = mapping[flagNum]
        
    
        try:
            # extract the fix for the given syndrome
            fix = fixer[syndrome]
            # implement it
            circuit_builder.buildCirc(qc, correction, perm)
        # This case probably happens but I'm not sure what to do with it yet
        except:
            print("Unexpected syndrome in fixFlag")
        
        
    
    def flagCorrect(self,qc,perm):
        # go through the measurements
        # if we get a flag then we're good, set a "flag code"
        # if we get some nontrivial syndrome measurement 
        # then measure syndrome and correct
        
        stab = -1
        syndrome = ""
        for i in range(4):
            m = scheme.measureStab(qc,i,True)
            print(m)
            synd = str(m[0])
            flag = str(m[1])
            if flag == str(1):
                stab = i
                break
            if synd == str(1):
                stab = i
                break
        if stab != -1:
            syndrome = scheme.unflagCorrect(qc,perm)
            print(syndrome)
        
    # careful with perm here
    # MAKE THIS FAULT TOLERANT LATER
    def measureOutcome(self,qc,perm):
        transH = ["CX 3 2", "CX 5 1", "CX 5 4", "CX 3 4",
                 "CX 2 4", "CX 1 4", "H 1 2 3 4 5", "CZ 1 5",
                 "CZ 2 3", "H 1 2", "CX 2 4", "CX 1 4"]
        
        circuit_builder.buildCirc(qc, transH, perm)
        
        allPlus = ["CZ 1 5", "CZ 4 5", "CZ 2 3", "CZ 3 4", 
                   "CZ 1 2", "H 1 2 3 4 5"]
            
        circuit_builder.buildCirc(qc,allPlus,perm)
        
        firstPerm = {}
        for i in range(n):
                firstPerm[n-i] = i
            
        registerPerm = {}
        for i in perm.keys():
            registerPerm[firstPerm[i]] = perm[i]
        
        for i in range(n):
            qc.measure(qr[registerPerm[i]],cr[i])
        
        job = execute(qc,backend)
        result = job.result()
        print(result.get_counts())
                

In [516]:
# Do all the actual benchmarking over here

scheme = FiveOneThree()

n = scheme.n
validOutputs = scheme.validOutputs
perm = scheme.perm
scheme.setPerm()

#qr = QuantumRegister(n)
#cr = ClassicalRegister(n)
#qc = QuantumCircuit(qr,cr)

qc1 = QuantumCircuit()

init = False

while not init:
    qc = QuantumCircuit()
    indicator = scheme.prepareZeros(qc)
    if indicator != "FAILURE":
        init = True
print("State Preparation Successful")

qr = getQReg(qc,0)  
cr = getCReg(qc,0)

scheme.setInput(qc,0)

scheme.unflagCorrect(qc,perm)

scheme.unflagCorrect(qc,perm)

scheme.measureOutcome(qc,perm)


State Preparation Successful

USED AN ANCILLA
Quantum registers...
OrderedDict([('q963', QuantumRegister(5, 'q963')), ('q964', QuantumRegister(1, 'q964'))])
Ancilla Register
QuantumRegister(1, 'q964')
1 ancillas used so far
Measuring stabilizer 0


Got a 400 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=MBtItFkyjksCBDeCphad6rpMFoGlu5S24wT6Jcv47BFj0GRBgzZBdyx8wGEXieOB: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 0. Gates after a measure are blocked","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=MBtItFkyjksCBDeCphad6rpMFoGlu5S24wT6Jcv47BFj0GRBgzZBdyx8wGEXieOB: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 0. Gates after a measure are blocked","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=MBtItFkyjksCBDeCphad6rpMFoGlu5S24wT6Jcv47BFj0GRBgzZBdyx8wGEXieOB: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 0. Gates after a measure are blocked","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://quantumexperie

QISKitError: '"{\'status\': 400, \'message\': \'Error parsing QASM. Error parsing qasm number 0. Gates after a measure are blocked\', \'code\': \'QASM_NOT_VALID\', \'statusCode\': 400}"'

In [523]:
backend = "ibmqx4"

r1 = QuantumRegister(3)
r2 = QuantumRegister(2)
c1 = ClassicalRegister(3)
c2 = ClassicalRegister(2)
q = QuantumCircuit(r1,r2,c1,c2)

q.x(r1)
q.h(r2)

q.measure(r1,c1)
q.barrier()

job = execute(q,backend)
result = job.result()
print(result.get_counts())

q.x(r2)

q.measure(r2,c2)

job = execute(q,backend)
result = job.result()
print(result.get_counts())


Got a 400 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=MBtItFkyjksCBDeCphad6rpMFoGlu5S24wT6Jcv47BFj0GRBgzZBdyx8wGEXieOB: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 0. Gates after a measure are blocked","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=MBtItFkyjksCBDeCphad6rpMFoGlu5S24wT6Jcv47BFj0GRBgzZBdyx8wGEXieOB: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 0. Gates after a measure are blocked","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://quantumexperience.ng.bluemix.net/api/Jobs?access_token=MBtItFkyjksCBDeCphad6rpMFoGlu5S24wT6Jcv47BFj0GRBgzZBdyx8wGEXieOB: {"error":{"status":400,"message":"Error parsing QASM. Error parsing qasm number 0. Gates after a measure are blocked","code":"QASM_NOT_VALID","statusCode":400}}
Got a 400 code response to https://quantumexperie

QISKitError: '"{\'status\': 400, \'message\': \'Error parsing QASM. Error parsing qasm number 0. Gates after a measure are blocked\', \'code\': \'QASM_NOT_VALID\', \'statusCode\': 400}"'